In [1]:
import sklearn
import pandas as pd
import numpy as np
import tensorflow as tf
import scipy

import sys
from PIL import Image
from os import listdir
from os.path import isfile, join
import os, errno
import csv

from rainforest_functions import vectorize_categories
from rainforest_functions import append_rotated_images

In [2]:
mypath = '/Users/AnaSolaguren-Beascoa/Pictures/train-jpg/'

In [4]:
#THIS PROGRAMME LOOPS 3 TIMES, ONE TIME FOR EACH RGB COLOUR

for x in range(3):
    print(x)
    #########################################################
    #open the files which contain the pixels
    #########################################################
    if x==0:
        pixel_df_0 = pd.read_csv(mypath+'pixels_R.csv')
        pixel_df_0 = pixel_df_0.rename(columns = {str(pixel_df_0.shape[1]-1):'image_name'})
        pixel_df = pixel_df_0
        del(pixel_df_0)


    if x==1:

        pixel_df_1 = pd.read_csv(mypath+'pixels_G.csv')
        pixel_df_1 = pixel_df_1.rename(columns = {str(pixel_df_1.shape[1]-1):'image_name'})
        pixel_df = pixel_df_1
        del(pixel_df_1)

    if x==2:
        pixel_df_2 = pd.read_csv(mypath+'pixels_B.csv')
        pixel_df_2 = pixel_df_2.rename(columns = {str(pixel_df_2.shape[1]-1):'image_name'})
        pixel_df = pixel_df_2
        del(pixel_df_2)


    #########################################################
    #for each color, store the pixels into a dataframe
    #and assign it the corresponding categories
    #########################################################
    num_files = pixel_df.shape[0]

    #read categories
    image_name_df = pd.read_csv('train_v2.csv')
    image_name_df_0 = image_name_df.copy()

    #split the categories
    categories = image_name_df_0['tags'].str.split(' ', expand=True).stack().unique()

    #merge dataframes
    result = pd.merge(pixel_df, image_name_df_0, on='image_name')
    del(pixel_df)

    #define X and y
    X = result.drop(['tags','image_name'],axis=1)
    y_all = result['tags']

    del(result)

    #vectorise the categories: e.g. convert 'agriculture' into [1,0,0,0,0,0,0,...,0]
    y_mine_all = vectorize_categories(categories, y_all)
    y_mine_all.columns = categories



    ##################################################################################
    # Here the programme rotates the images for each color
    ##################################################################################

    colors = ['R','G','B']

    ycol = list(y_mine_all.columns)

    #repeat for each category
    for k in range(len(ycol)):

        print(categories[k])
        
        #if files exist, delete
        try:
            os.remove(mypath+'all_rotated/final_pixels_'+categories[k]+'_'+colors[x]+'.csv')
        except OSError:
            pass
        
        
        ##################################################################################
        #Open the file and write each pixel on a line
        ##################################################################################
        with open(mypath+'all_rotated/final_pixels_'+categories[k]+'_'+colors[x]+'.csv', 'a') as csvfile:
            writer=csv.writer(csvfile, delimiter=',',lineterminator='\n')

            #Write the corresponding column names
            writer.writerow(list(range(X.shape[1]))+categories.tolist())


            ##################################################################################
            #count the numbers of 0s and 1s for each class. This measures the asymmetry 
            #and how many times a picture must be rotated
            ##################################################################################
            count0 = y_mine_all[y_mine_all[ycol[k]]==0][ycol[k]].count()
            count1 = y_mine_all[y_mine_all[ycol[k]]==1][ycol[k]].count()

            #############################################################################################
            #depending on which is the largest count, add new rotated images to symmetrise the data
            #############################################################################################
            
            #if more 0s than 1s
            if (count0>count1):

                y_single = pd.DataFrame(y_mine_all[ycol[k]])
                y_1_single = y_single[y_single[ycol[k]]==1]                              
                y_1 = y_mine_all[y_mine_all[ycol[k]]==1]  
                X_1 = X.loc[list(y_1_single.index)]
                asymmetry = int(count0/count1)
                for i in range(len(X_1)-1):
                    X_1_line = X_1.iloc[i].as_matrix()
                    y_1_line = y_1.iloc[i].as_matrix()
                    
                    #this function rotates each image asymmetry times 
                    X_rot_one, y_rot_one = append_rotated_images(X_1_line, y_1_line, asymmetry)
                    if i%100==0:
                        print('.', end='', flush=True)
                        if i%1000==0 and i!=0:
                            print(i)
                            
                    #write rotated pixels to file
                    for j in range(X_rot_one.shape[0]):
                        pixels_flat = X_rot_one.iloc[j].as_matrix()
                        writer.writerow(np.append(pixels_flat, y_rot_one.iloc[j]))


                del(y_single)
                del(y_1_single)
                del(X_1)
                del(y_1)

            #if more 1s than 0s
            if (count0<count1):
                y_single = pd.DataFrame(y_mine_all[ycol[k]])
                y_0_single = y_single[y_single[ycol[k]]==0]                              
                y_0 = y_mine_all[y_mine_all[ycol[k]]==0]  
                X_0 = X.loc[list(y_0_single.index)]
                asymmetry = int(count1/count0)
                for i in range(len(X_0)-1):
                    X_0_line = X_0.iloc[i].as_matrix()
                    y_0_line = y_0.iloc[i].as_matrix()
                    
                    #this function rotates each image asymmetry times 
                    X_rot_one, y_rot_one = append_rotated_images(X_0_line, y_0_line, asymmetry)
                    if i%100==0:
                        print('.', end='', flush=True)
                        if i%1000==0 and i!=0:
                            print(i)
                            
                    #write rotated pixels to file
                    for j in range(X_rot_one.shape[0]):
                        pixels_flat = X_rot_one.iloc[j].as_matrix()
                        writer.writerow(np.append(pixels_flat, y_rot_one.iloc[j]))
                del(y_single)
                del(y_0_single)
                del(X_0)
                del(y_0)

    del(X)
    del(y_mine_all)
print('END')

0
haze
.0
..........1000
..........2000
......primary
agriculture
.0
..........1000
..........2000
..........3000
..........4000
..........5000
..........6000
..........7000
..........8000
..........9000
..........10000
..........11000
..........12000
...clear
water
.0
..........1000
..........2000
..........3000
..........4000
..........5000
..........6000
..........7000
....habitation
.0
..........1000
..........2000
..........3000
......road
.0
..........1000
..........2000
..........3000
..........4000
..........5000
..........6000
..........7000
..........8000
cultivation
.0
..........1000
..........2000
..........3000
..........4000
....slash_burn
.0
..cloudy
.0
..........1000
..........2000
partly_cloudy
.0
..........1000
..........2000
..........3000
..........4000
..........5000
..........6000
..........7000
..conventional_mine
.0
bare_ground
.0
........artisinal_mine
.0
...blooming
.0
...selective_logging
.0
...blow_down
.0
1
haze
.0
..........1000
..........2000
......primar